# Object-Relational Mapping (ORM)

### SQLAlchemy

- http://docs.sqlalchemy.org/en/latest/orm/tutorial.html
- `pip3 install sqlalchemy`
- python에서 사용하는 ORM

ORM이란?
- Object-relational mapping
- 데이터 베이스를 객체화 시켜 데이터베이스에 있는 데이터를 CRUD를 할수 있음.
- 사용하는 DB를 변경해야 할 경우 데이터 베이스 엔진만 바꾸면 됨

>- cursur가 아닌 session(함수)를 실행해서 데이터를 주고 받을 수 있음 (query를 실행하는 것이 아니다)
>- 데이터 베이스의 테이블을 객체화.
>- 객체를 함수를 통해서 보낸다.

- SQLAlchemy에서는 함수를 유지하고 엔진만 바꿔서 데이터베이스를 변경할 수 있다.


1. db connect
2. mapping class (db.table - class)
3. insert
4. select
5. update
6. delete

In [18]:
import sqlalchemy, pickle
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, DateTime
from sqlalchemy.orm import sessionmaker
import MySQLdb
import pandas as pd

##### db connect

In [19]:
# mysql+mysqldb://<사용자계정>:<비밀번호>@<호스트>/<데이터베이스_이름>
# pw = pickle.load(open('pw.p','rb'))
engine = sqlalchemy.create_engine("mysql+mysqldb://root:" + "110391" + "@127.0.0.1/test_alchemy")

##### mapping class

In [22]:
# __tablename__ : 데이터베이스의 테이블 이름

Base = declarative_base()
class User(Base):
    __tablename__ = 'user2'

    user_id = Column(Integer, primary_key=True)
    name = Column(String(16))
    email = Column(String())
    age = Column(Integer)
    rdate = Column(DateTime)
    
    def __init__(self, name, email, age, rdate):
        self.name = name
        self.email = email
        self.age = age
        self.rdate = rdate
    
    def __repr__(self):
        return "<User {}, {}, {}, {}>".format(self.name, self.email, self.age, self.rdate)

In [23]:
# engine에 연결된 데이터 베이스 테이블 생성 (연결, Mapping 실행)
Base.metadata.create_all(engine)

CompileError: (in table 'user2', column 'email'): VARCHAR requires a length on dialect mysql

In [8]:
# 세션 생성 (sessionmaker기능)
Session = sessionmaker(bind=engine)
session = Session()

##### insert

In [19]:
# add user object to session
# add one object
# 하나의 데이터를 넣어줄 때는 add.
# 하나의 데이터를 객체로 만들어서 집어 넣어준다
user = User('jin', 'jin@gmail.com', '27', '2016-03-21')
session.add(user)

In [15]:
# 여러 개의 데이터를 넣어줄 때는 리스트 형태로
# add multy objects
users = [
    User('alice', 'alice@gmail.com', '25', '2018-02-21'),
    User('andy', 'andy@daum.net', '33', '2015-01-19'),
]
session.add_all(users)

In [20]:
# run transaction (세선의 object가 insert)
session.commit()

In [21]:
# 롤백 (commit 하기 전에, commit 전의 상태에서 session을 초기화 하고 싶을 때)
# rollback (session data clear)
session.rollback()

##### select
- filter
- order by

In [22]:
# list 형태의 object 안의 요소를 하나씩 꺼내는 함수

def disp(datas):
    for data in datas:
        print(data)

In [23]:
results = session.query(User).all()
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [29]:
# filter
# ==, !=, >, <, >=, <=, like, in_, ~(not. 제외한 나머지 데이터 모두), 
results = session.query(User).filter(User.name == "jin")
disp(results)

<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [14]:
results = session.query(User).filter(User.name != "jin")
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [30]:
results = session.query(User).filter(User.age > 26)
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [31]:
# like
results = session.query(User).filter(User.email.like("%gmail%"))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [32]:
# in_
results = session.query(User).filter(User.name.in_(["alice","andy"]))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [41]:
# not
results = session.query(User).filter(~User.email.like("%gmail%"))
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [34]:
# and_, or_
from sqlalchemy import and_, or_
results = session.query(User).filter(or_(User.name == "jin", User.age == 33))
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [39]:
results = session.query(User).filter(and_(User.age < 30, User.email.like("%gmail%")))
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>


In [42]:
# order_by
results = session.query(User).order_by(User.age.asc())
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [37]:
results = session.query(User).order_by(User.age)
disp(results)

<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>


In [43]:
results = session.query(User).order_by(User.age.desc())
disp(results)

<User andy, andy@daum.net, 33, 2015-01-19 00:00:00>
<User jin, jin@gmail.com, 27, 2016-03-21 00:00:00>
<User alice, alice@gmail.com, 25, 2018-02-21 00:00:00>


In [44]:
# count
session.query(User).count()

3

##### update

In [50]:
# one : 하나의 데이터를 가져온다
# first : 가장 첫 데이터를 가져온다
jin = session.query(User).filter(User.name == "jin").one()

In [51]:
jin.age = 30

In [52]:
session.add(jin)

In [53]:
session.commit()

##### delete

In [54]:
session.query(User).filter(User.name == "jin").delete()

1

In [55]:
session.commit()

In [56]:
# close session
session.close()